<a href="https://colab.research.google.com/github/TetianaHrunyk/NeuralMetaphorsDetection/blob/master/MD4_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface-hub==0.0.8

In [ ]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 245kB 26.2MB/s 
     |████████████████████████████████| 2.3MB 29.1MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 245kB 35.2MB/s 
     |████████████████████████████████| 122kB 53.3MB/s 
     |████████████████████████████████| 901kB 33.4MB/s 
     |████████████████████████████████| 3.3MB 33.1MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=d9124ee477b720622f8377b7b5401abd865be16e55acb2b5f3fa3544208e11c3
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [ ]:
data_path='https://raw.githubusercontent.com/TetianaHrunyk/NeuralMetaphorsDetection/master/'

In [ ]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from torch.utils.data import Dataset
import pandas as pd
import torch
import torch.functional as nn
from torch.utils.data import DataLoader

# Preprocessing the data

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, path, nrows=None, skiprows=None):
    df = pd.read_csv(path, header=None, names=["txt"], dtype='str', nrows=nrows, skiprows=skiprows)
    df.dropna(inplace=True)
    df.txt = df.txt.str.split(" ")
    self.labels = list(map(lambda x: [int(word.startswith("M_")) for word in x], df.txt))
    self.text = list(map(lambda x: [ word.replace("M_", "") for word in x], df.txt))
    self.rows = len(self.text)
    self.data, self.new_labels = self.tokenize_and_align_labels(self.labels, self.text)

  def __len__(self):
    return self.rows

  def __getitem__(self, idx):
#    assert len(self.data[idx].attention_mask) == len(self.data[idx].ids) == len(self.new_labels[idx])
    return {'attention_mask': self.data[idx].attention_mask,
            'input_ids': self.data[idx].ids,
            'labels': self.new_labels[idx],
#            'tokens': self.text[idx]
            }

  def tokenize_and_align_labels(self, old_labels, text, label_all_tokens=False):
    tokenized_inputs = tokenizer(text, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(old_labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    return tokenized_inputs, labels

In [ ]:
lines = pd.read_csv(data_path+"McMillanMetaphorsAll_train.csv").shape[0]
train_data = CustomDataset(data_path+"McMillanMetaphorsAll_train.csv", nrows=round(lines*0.9))
val_data = CustomDataset(data_path+"McMillanMetaphorsAll_train.csv",  skiprows=round(lines*0.9))
test_data = CustomDataset(data_path+"McMillanMetaphorsAll_test.csv")


In [ ]:
train_data[1]

{'attention_mask': [1, 1, 1, 1, 1, 1],
 'input_ids': [101, 7004, 7459, 7967, 1012, 102],
 'labels': [-100, 0, 0, 0, -100, -100]}

# Fine-tuning the model

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [ ]:
args = TrainingArguments(
    "MetahporsDetection",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from datasets import  load_metric
metric = load_metric("seqeval")

In [ ]:
import numpy as np
label_list = ["Literal", "Metaphorical"]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.012894,0.985262,0.984724,0.984993,0.997606
2,0.061900,0.001060,1.000000,1.000000,1.000000,1.000000
3,0.004800,0.000261,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=1176, training_loss=0.028645704620752204, metrics={'train_runtime': 3576.7112, 'train_samples_per_second': 0.329, 'total_flos': 20896827939840.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 780435456, 'train_mem_cpu_peaked_delta': 4456448})

# Evaluate model

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_loss': 0.0002608223876450211,
 'eval_mem_cpu_alloc_delta': 212992,
 'eval_mem_cpu_peaked_delta': 57344,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_runtime': 31.2625,
 'eval_samples_per_second': 22.295}

In [ ]:
predictions, labels, _ = trainer.predict(val_data)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'etaphorical': {'f1': 1.0, 'number': 571, 'precision': 1.0, 'recall': 1.0},
 'iteral': {'f1': 1.0, 'number': 1262, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [ ]:
results['etaphorical']

In [ ]:
for rkey, rval in results.items():
  if type(rval) == dict:
    if rkey == 'etaphorical':
        print('Metaphorical')
    if rkey == 'iteral':
        print('Literal')
    for key, val in rval.items():
      
      print("{:18} : {}".format(key, round(val*100, 2)))
    print()
    continue
  print("{:18} : {}".format(rkey, round(rval*100, 2)))

Metaphorical
precision          : 100.0
recall             : 100.0
f1                 : 100.0
number             : 57100

Literal
precision          : 100.0
recall             : 100.0
f1                 : 100.0
number             : 126200

overall_precision  : 100.0
overall_recall     : 100.0
overall_f1         : 100.0
overall_accuracy   : 100.0


# Test the model

In [ ]:
def predict(sent, label_all_tokens=False):
    tokenized = tokenizer(sent, is_split_into_words=False, truncation=True)
    tokens = tokenizer.convert_ids_to_tokens(tokenized.input_ids)
    predictions, _, _ = trainer.predict([tokenized])
    labels = np.argmax(predictions[0], axis=1)
    for token, label in zip(tokens[1:-1], labels[1:-1]):
      print("{:15} : {:15}".format(token, label_list[label]))


In [ ]:
predict("Mechanical orange")

In [ ]:
predict("car drinks gasoline")

In [ ]:
predict("kids have been screaming loudly outside")

In [ ]:
predict("I pushed myself to the edge")

In [ ]:
predict("Chicken or egg first dilemma")

In [ ]:
predict("moderate intake of eggs is good for our health")

# Save model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/MsThesis/TrainedModels/fineTunedBERT_mixed.txt')

# Load Model

In [ ]:
#trainer.model.load_state_dict(torch.load('/content/drive/MyDrive/MsThesis/TrainedModels/fineTunedBERT.txt'))
#trainer.model.eval()

In [ ]:
#predict("Some data")